# Make gif of images as camera sweeps 

In [ ]:
import sys
import os
from os.path import join
import numpy as np
import pandas as pd
import h5py
import imageio
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from ipywidgets import interactive
from ipywidgets import widgets
from matplotlib import pyplot as plt

sys.path.append('../..')
from tools import utils

In [ ]:
datadir = '/Diagnostics/Data/Measurements/scan-xxpy-image-ypdE/2021-12-03/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = 'merged2-211203151919-211203192701-scan-xxpy-image-ypdE'
file = h5py.File(join(datadir, filename + '.h5'), 'r')
print(list(file))

In [ ]:
data = file['scandata']

In [ ]:
acts = ['y_PositionSync', 'xp_PositionSync', 'x_PositionSync']

In [ ]:
shape = (258, 346)

In [ ]:
images = data['cam06_Image'].reshape((len(data), shape[0], shape[1]))

In [ ]:
images.shape

In [ ]:
imax = np.argmax(np.sum(images, axis=(1, 2)))

In [ ]:
imax

In [ ]:
iterations = data['iteration'][:]

In [ ]:
max_iteration = 0
for iteration in np.unique(iterations):
    idx, = np.where(iterations == iteration)
    if imax in idx:
        max_iteration = iteration
        break

In [ ]:
max_iteration

In [ ]:
_iterations = np.arange(max_iteration, max_iteration + 2)
inds = []
for _iteration in _iterations:
    idx, = np.where(iterations == _iteration)
    inds.extend(idx)

In [ ]:
ys = data[inds, 'y_PositionSync']
x2s = data[inds, 'xp_PositionSync']
x1s = data[inds, 'x_PositionSync']

In [ ]:
ims = images[inds]

In [ ]:
from matplotlib import animation

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4), constrained_layout=True)
plt.close()

vmin = np.min(ims)
vmax = np.max(ims)
# vmin = vmax = None

def update(i):
    im = ims[i]
    ax.clear()
    ax.set_title('')
    # ax.set_xlabel('x3')
    # ax.set_ylabel('y3')
    ax.set_yticks([])
    ax.set_xticks([])
    for text in ax.texts:
        text.set_visible(False)
    ax.pcolormesh(im, cmap='viridis', shading='auto', ec='None',
                  vmin=vmin, vmax=vmax)
    ax.annotate(
        f"y1 = {ys[i]:.2f},  x2 = {x2s[i]:.2f},  x1 = {x1s[i]:.2f}",
        xy=(0.5, 0.98), xycoords='axes fraction',
        horizontalalignment='center', verticalalignment='top',
        color='white', fontsize=12,
    )

anim = animation.FuncAnimation(
    fig, update, frames=len(ims), blit=False,
)
anim.save('sweeps.gif', dpi=350, fps=10)

In [ ]:
fig, (axt, ax) = plt.subplots(
    nrows=2, ncols=1, figsize=(4, 4.5), constrained_layout=True,
    gridspec_kw=dict(height_ratios=[0.1, 1.0]),
)
axt.set_yticks([])
axt.set_xlim(np.min(ys), np.max(ys))
axt.set_xlabel(r"$y_1$ [mm]", fontsize=11)
for side in ['top', 'right', 'left']:
    axt.spines[side].set_visible(False)

line = axt.axvline(color='black')
plt.close()

def update(i):
    im = ims[i][::-1, :]
    ax.clear()
    ax.set_title('')
    ax.set_yticks([])
    ax.set_xticks([])
    for text in ax.texts:
        text.set_visible(False)
    ax.imshow(im[25:-25, 50:-50], vmin=vmin, vmax=vmax)
    ax.annotate(
        f"$x_2$ = {x2s[i]:.2f} [mm],  $x_1$ = {x1s[i]:.2f} [mm]",
        xy=(0.02, 0.98), xycoords='axes fraction',
        horizontalalignment='left', verticalalignment='top',
        color='white', fontsize=12,
    )
    line.set_data((ys[i], ys[i]), (0.0, 1.0))
    return ax

anim = animation.FuncAnimation(
    fig, update, frames=len(ims), blit=False,
)
anim.save('sweeps.gif', dpi=350, fps=10)